In [ ]:
'''
Get the transaction history of the ERC20 tokens. 
Author: Jinhua Wang
License: MIT
Powered by Etherscan.io APIs
'''

In [ ]:
from bs4 import BeautifulSoup
import urllib3
#disable the annoying security warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import sys
import json
import collections
import pandas as pd
#the rate control module
import ratel

In [ ]:
#the base url for etherscan api
baseUrl='https://api.etherscan.io'
#the connection pool
pool=urllib3.PoolManager(5)
#number of threads
numthread=5

#get the API key
key=''
keyarr=[]
keyfile='../Credentials/etherscankey.txt'
with open(keyfile) as f:
    keyarr = f.readlines()
if len(keyarr)==0 or keyarr[0] == '':
    print "API Key Not Found in Directory! Aborting ..."
    sys.exit(0)
key=keyarr[0]

#limit the rate that the api could be called (5 times per second)
@ratel.rate_limited(5,1)
def call_api(url):
    '''
    Make API calls 
    '''    
    global pool
    r=pool.request('GET',url)
    if r.status != 200:
        raise ApiError('Cannot call API: {}'.format(response.status_code))
    return r

#convert dict items to strings
def convert(data):
    if isinstance(data, basestring):
        return str(data)
    elif isinstance(data, collections.Mapping):
        return dict(map(convert, data.iteritems()))
    elif isinstance(data, collections.Iterable):
        return type(data)(map(convert, data))
    else:
        return data

def getHistory(fromBlock, toBlock, contractAdd, topic):
    global key
    r=call_api("https://api.etherscan.io/api?module=logs&action=getLogs&fromBlock=3957992&toBlock=10000000&address=0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0&topic0=0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef&apikey=HCVQ8D297HTGK44N7J3GSVC5XV753ZHRUY")
    html=r.data
    #remove the escaped string
    o=json.loads(html)
    o=convert(o)
    df=pd.DataFrame.from_dict(o["result"])
    for index, row in df.iterrows():
        print row["topics"][0]
    
getHistory(1,2,3,4)